# Une analyse qui fleure bon les années 50

## Introduction

Cette étude de cas est extraite de l'ouvrage de Jean-Pierre Benzécri
"Analyse des données. Tome 2 : l'analyse des correspondances". Il
s'agit d'une étude réelle dont nous avons complété les
objectifs. 

## But

Une enquête a été effectuée auprès de cent fumeurs afin de choisir les
noms de deux nouvelles marques de cigarettes. La première marque est
destinée à une clientèle masculine : L'homme ciblé est un connaisseur
distingué, raffiné mais viril, de niveau socio-économique élevé. La
seconde symbolise un public féminin, élégant, assuré, dynamique.

## Les donées

Douze marques ont été retenues :
- Orly (Orl)
- Alezan (Ale)
- Corsaire (Cor)
- Directoire (Dir)
- Ducat (Duc)
- Fontenoy (Fon)
- Icare (Ica)
- Zodiac (Zod)
- Pavois (Pav)
- Cocker (Coc)
- Escale (Esc)
- Hôtesse (Hot)

Pour évaluer leur image auprès du public, onze attributs ont été
proposés aux fumeurs :
- vieillot - désuet (VD)
- nouveau riche (NR)
- sobre, élégant (SE)
- cocasse - ridicule (CR)
- racé (RA)
- mièvre (MI)
- distingué (DI)
- vulgaire - commun (VC)
- pour un homme (HO)
- pour une femme (FE)
- pour une petite nature (PN)

On a demandé à chacun des cent fumeurs d'associer une ou plusieurs
parques qui selon eux conviennent le mieux.
Le tableau de données est disponible dans le fichier "fume.txt".

In [ ]:
import pandas

In [ ]:
fume = pandas.read_csv("fume.txt", delim_whitespace=True)

In [ ]:
fume

## Analyse

### Quelles sont les marques les plus citées ? Attributs ?

In [ ]:
fume.sum().sort_values(ascending=False)

In [ ]:
fume.sum(axis=1).sort_values(ascending=False)

- Attributs en commun : "SE", "RA" et "DI".
- On propose des attributs qui ne leur correspondent pas pour ne garder que les marques ayant des aspects positifs.

## Regarder la correlation entre les marques. Quelles sont les marques dont les correlations sont les plus faibles ? Les plus élevées ? Comment interpréter ces résultats ?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
current_palette = sns.color_palette()

In [ ]:
data = fume.corr().applymap(abs)

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=0, vmax=1)
plt.show()

## Les publics visés ont-ils des points communs ? Quels sont les attributs correspondants  Pourquoi a-t-on proposé des attributs ne leur correspondant pas ?

* Attributs en commun : `SE`, `RA` et `DI`
* On propose des attributs qui ne leur correspondent pas pour ne garder que les marques ayant des aspects positifs

## Regarder la correlation entre les attributs. Quelles sont les marques dont les correlations sont les plus faibles ? Les plus élevées ? Comment interpréter ces résultats ?

In [ ]:
data = fume.transpose().corr().applymap(abs)

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=0, vmax=1)
plt.show()

In [ ]:
data = fume.transpose().corr()

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data, 
            annot=True, 
            cmap='coolwarm',
            vmin=0, vmax=1)
plt.show()

* `HO`corrélé avec `VC` (en positif) et `MI` (en négatif) : attention lors des interprétations ! On veut juste une marque pour un homme pas pour un plouc (il faudra trouver des axes qui séparent ces deux attributs)
* `FE`très peu corrélé avec `NR`

## En examinant les valeurs propres et leur diagramme, déterminer les axes principaux qu'il faut garder dans les analyses.

Attention, nous voulons expliquer les marques par rapports aux attributs. Nos données sont donc les marques. Il faut transposer le tableau de données.

In [ ]:
fume.transpose()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = pandas.DataFrame(scaler.fit_transform(fume.transpose()),
                     columns=fume.transpose().columns, 
                     index=fume.transpose().index)

In [ ]:
from sklearn.decomposition import PCA 
import numpy as np

pca = PCA() 
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

corrélations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
cos2 = (C**2).div((X**2).sum(axis=1), axis='index')

In [ ]:
I

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

I.plot(ax=ax)

plt.title("inertie expliquée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquée cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

On garde tous les axes avant la "chute" de l'inertie expliquée (le gain d'un nouvel axe ne fait plus gagner grand chose) ou lorsque le pourcentage d'inertie expliquée est suffisant. Ici 5 axes nous permettent d'expliquer plus de 90% des données et l'axe 5 correspond à une chute d'inertie expliquée.

## Interprétation

### Les axes

In [ ]:
corrélations.transpose()[0:5]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) 

corrélations.transpose()[0:5].plot.bar(ax=ax)

plt.show()

In [ ]:
I.iloc[:5]

- composante 0 : 28% de l'inertie. Oppose vulgaire-commun (`VC`) à sobre-éléguant (`SE`), distingué (`DI`) même si avec une petite tendance nouveau riche (`NR`). Nos marques cibles sont à gauche de l'axe 0
- composante 1 : 21% de l'inertie. Oppose petite nature (`PN`) et cocasse-ridicule (`CR`) à Homme (`HO`). Axe de virilité (rappelez vous que c'est les années 50) : axe de *virilité* ?
- composante 2 : 14 % de l'inertie. Les Femmes (`FE`) (a gauche). *Opposition homme/femme* ?
- composante 3 : 13% de l'inertie. Racé (`RA`) (à droite)
- composante 4 : 11% de l'inertie. vieillot-désuet (`VD`) (à droite)

### Qualité de la projection

In [ ]:
cos2[list(range(5))]

In [ ]:
axe_x = 0
axe_y = 3

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

Nos marques cible sont à gauche. Le deuxième axe (raçé) est plus typé homme.

In [ ]:
axe_x = 0
axe_y = 2

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

Hotesse est très typé femme et Alezan typé Homme. Nos choix se portent là dessus.

In [ ]:
axe_x = 0
axe_y = 1

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show()

Directoire n'est pas très viril alors qu'Hotesse, oui.

In [ ]:
axe_x = 0
axe_y = 4

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

Alezan se projette mal sur l'axe 4, mais pavois et Orly font très nouveau riche.